Урок 4. Система управления базами данных MongoDB в Python
1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

In [100]:
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
from time import sleep

In [106]:
URL_HH = 'https://spb.hh.ru/search/vacancy'

In [107]:
HEADER = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}

In [108]:
def get_html(url, params=''):
    response = requests.get(url, headers = HEADER, params = params)
    
    return response

response = get_html(URL_HH, params={'text': 'Разработчик Python', 'page': 0})

In [109]:
def get_hh_content(response):
    soup = bs(response.text, 'html.parser')
    items = soup.find_all('div', {'class': 'serp-item'})
    
    vacancy = []
    for item in items:
        title = item.find('a', {'class': 'serp-item__title'}).get_text()
        link = item.find('a', {'class': 'serp-item__title'})['href']
        city = item.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).contents[0].strip(', ')
        company = item.find('a', {'class': 'bloko-link_kind-tertiary'}).get_text()
        try:
            salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).get_text()
        except:
            salary = ''

        vacancy.append(
            {'title': title,
             'link': link,
             'city': city,
             'company': company,
             'salary': salary,
             'site': 'HeadHunter'})
        
    return vacancy

get_hh_content(response)

[{'title': 'Go разработчик со знаниями Python',
  'link': 'https://spb.hh.ru/vacancy/77799276?from=vacancy_search_list&query=%D0%A0%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA+Python',
  'city': 'Алматы',
  'company': 'ТОО\xa0Freedom Drive',
  'salary': 'до 1\u202f000\u202f000 KZT',
  'site': 'HeadHunter'},
 {'title': 'Team Lead Python backend (удаленно)',
  'link': 'https://spb.hh.ru/vacancy/77690655?from=vacancy_search_list&query=%D0%A0%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA+Python',
  'city': 'Москва',
  'company': 'Франчайзинговая розничная сеть Галамарт',
  'salary': 'от 300\u202f000 руб.',
  'site': 'HeadHunter'},
 {'title': 'Разработчик курсов IT и ИБ',
  'link': 'https://spb.hh.ru/vacancy/77713311?from=vacancy_search_list&query=%D0%A0%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA+Python',
  'city': 'Москва',
  'company': 'ООО\xa0Кодебай',
  'salary': 'от 300\u202f000 руб.',
  'site': 'HeadHunter'},
 {'title': 'Преподаватель Pyt

In [110]:
def edit_data_hh(vacancy):
    for vac in vacancy:
        # Del \xa0
        vac['company'] = vac['company'].replace(u'\xa0', u' ')
        vac['salary'] = vac['salary'].replace(u'\u202f', u'')

        # Salary (min, max, currency)
        if vac['salary']:
            salary_list = vac['salary'].split(' ')
            if salary_list[0] == 'от':
                vac['salary_min'] = float(salary_list[1])
                vac['salary_max'] = None
            elif salary_list[0] == 'до':
                vac['salary_min'] = None
                vac['salary_max'] = float(salary_list[1])
            else:
                vac['salary_min'] = float(salary_list[0])
                vac['salary_max'] = float(salary_list[2])
            vac['salary_currency'] = salary_list[-1]
        else:
            vac['salary_min'] = None
            vac['salary_max'] = None
            vac['salary_currency'] = None
        vac.pop('salary')

        # Id
        id = vac['link'].split(sep='/')[4].split(sep='?')[0]
        vac['_Id'] = id
    
    return vacancy

In [ ]:
def parser_hh():
    post = str(input('Введите название вакансий для парсинга: '))
    pages = int(input('Введите количество страниц для парсинга: '))
    
    result = []
    for i in range(pages):
        print(f'Парсится hh, страница: {i + 1}')
        response = get_html(URL_HH, params = {'text': post, 'page': i})
        vacancy = get_hh_content(response)
        vacancy_edit = edit_data_hh(vacancy)
        if response.ok and vacancy:
            result.extend(vacancy_edit)
        else:
            break

    return result

result = parser_hh()

In [92]:
import pymongo
from pymongo import MongoClient
import pymongo.errors

In [93]:
client = MongoClient('mongodb://localhost:27017/')
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [94]:
db = client.headhunter
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'headhunter')

In [95]:
vacancies = db.vacancies
vacancies

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'headhunter'), 'vacancies')

In [ ]:
def insert_mongo(result):
    for vac in result:
        try:
            vacancies.update_one({'_Id': vac['_Id']}, {'$set': vac}, upsert = True)
        except idoc:
            print(f'Error {idoc}')
            continue

insert_mongo(result)

In [ ]:
cursor = vacancies.find()

for record in cursor:
    pprint(record)

In [ ]:
cursor = len(list(vacancies.find()))
print('Количество документов в коллекции:', cursor)

2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля).

In [ ]:
def search_by_salary():
    value = int(input('Введите максимальное значение заработной платы: '))   
    res = vacancies.find({'$or': [{'salary_currency': 'руб.',
                                   '$or': [{'salary_min': {'$gt': value}},
                                           {'salary_max': {'$gt': value}},
                                          ]},
                                  {'salary_currency': 'USD',
                                   '$or': [{'salary_min': {'$gt': value / 75}},
                                           {'salary_max': {'$gt': value / 75}},
                                          ]
                                 }]
                        })
    
    return (list(res))

res = search_by_salary()
pprint(res)

In [ ]:
print('Количество найденных вакансий, удовлетворяющих условию задания:', len(list(res)))